In [21]:
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
import PIL

In [22]:
PIL.Image.MAX_IMAGE_PIXELS = 933120000

In [23]:
# Define the paths to your data directories
train_dir = 'EyeContact/train'
validation_dir = 'EyeContact/validation'
test_dir = 'EyeContact/test'

In [24]:
# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

In [25]:
# Create a data generator with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 110 images belonging to 2 classes.


In [26]:
# Create data generators for validation and test data (without data augmentation)
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 14 images belonging to 2 classes.
Found 14 images belonging to 2 classes.


In [27]:
# Load the VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [28]:
# Freeze the layers of the VGG16 model
for layer in base_model.layers:
    layer.trainable = False

In [29]:
# Add custom layers for your specific classification task
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Linear activation for regression

In [30]:
# Create the final model
model = Model(base_model.input, output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/10
4/4 [==============================] - 196s 48s/step - loss: 1.0219 - accuracy: 0.4091 - val_loss: 0.6438 - val_accuracy: 0.6429
Epoch 2/10
4/4 [==============================] - 166s 48s/step - loss: 0.8181 - accuracy: 0.4545 - val_loss: 0.8731 - val_accuracy: 0.4286
Epoch 3/10
4/4 [==============================] - 182s 54s/step - loss: 0.7317 - accuracy: 0.5364 - val_loss: 0.6226 - val_accuracy: 0.6429
Epoch 4/10
4/4 [==============================] - 179s 45s/step - loss: 0.7620 - accuracy: 0.5182 - val_loss: 0.7007 - val_accuracy: 0.4286
Epoch 5/10
4/4 [==============================] - 179s 52s/step - loss: 0.6370 - accuracy: 0.6455 - val_loss: 0.7496 - val_accuracy: 0.4286
Epoch 6/10
4/4 [==============================] - 182s 42s/step - loss: 0.6789 - accuracy: 0.6455 - val_loss: 0.6369 - val_accuracy: 0.5714
Epoch 7/10
4/4 [==============================] - 174s 39s/step - loss: 0.7759 - accuracy: 0.4818 - val_loss: 0.6636 - val_accuracy: 0.5000
Epoch 8/10
4/4 [====

In [31]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy}')

1/1 [==============================] - 21s 21s/step - loss: 0.7527 - accuracy: 0.2857
Test Accuracy: 0.2857142984867096


In [32]:
# Save the model
model.save('models/eye_contact_model.h5')

I have videos of job interviews and I want to build a convolutional neural network that can classify whether the interviewee is maintaining ample eye contact or not. My data directory contains two sub-directories: "positive" which contains videos where ample eye contact is being maintained and "negative" which contains videos where ample eye contact is not being maintained.

Data preprocessing
1. The first step in our pipeline is to extract frames from the video at regular intervals. We use the get_frames() function to extract a frame every 3 seconds. 
2. Next, we use the Haar Cascade algorithm [17] to detect faces in each frame. 
3. Once we have located the face, we use the save_cropped() function to crop the face and save it for further processing.
